In [152]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler # 원-핫 인코딩, 표준 스케일링
from sklearn.compose import ColumnTransformer                   # 여러 컬럼에 다른 전처리 적용


In [153]:
car_df = pd.read_csv('./data/Dubizzle_used_car_sales.csv')
num_unique_including_nan = car_df['model'].unique().size
print(f"NaN을 포함한 'motors_trim'의 고유 값 개수 (unique().size): {num_unique_including_nan}")


NaN을 포함한 'motors_trim'의 고유 값 개수 (unique().size): 552


In [154]:
car_df
car_df =car_df.drop(['title','motors_trim','model','steering_side'], axis=1)

In [ ]:
print("\n--- 'horsepower' 컬럼 전처리 직전 상태 ---")
print(car_df['horsepower'].head(15)) # 상위 15개 값 확인
print(car_df['horsepower'].info())   # 데이터 타입 및 Non-Null 개수 확인
print("전처리 직전 고유 값:", car_df['horsepower'].unique()) # 모든 고유 값 확인 (특히 NaN이 어떻게 표현되는지)
print("전처리 직전 NaN 값 개수:", car_df['horsepower'].isna().sum()) # NaN 개수 확인


--- 'horsepower' 컬럼 전처리 직전 상태 ---
0              Unknown
1         400 - 500 HP
2         400 - 500 HP
3         600 - 700 HP
4     Less than 150 HP
5         200 - 300 HP
6              Unknown
7              Unknown
8         200 - 300 HP
9         200 - 300 HP
10        500 - 600 HP
11        500 - 600 HP
12        300 - 400 HP
13    Less than 150 HP
14             Unknown
Name: horsepower, dtype: object
<class 'pandas.core.series.Series'>
RangeIndex: 9970 entries, 0 to 9969
Series name: horsepower
Non-Null Count  Dtype 
--------------  ----- 
9970 non-null   object
dtypes: object(1)
memory usage: 78.0+ KB
None
전처리 직전 고유 값: ['Unknown' '400 - 500 HP' '600 - 700 HP' 'Less than 150 HP' '200 - 300 HP'
 '500 - 600 HP' '300 - 400 HP' '150 - 200 HP' '700 - 800 HP'
 '800 - 900 HP' '900+ HP']
전처리 직전 NaN 값 개수: 0


In [156]:

y = car_df['price_in_aed'] # 'price_in_aed'를 y로 정의

In [ ]:
import re

print("--- 전처리 전 car_df['horsepower'] 상태 ---")
print(car_df['horsepower'].head(15))
print(car_df['horsepower'].info())


# --- 1. horsepower 문자열을 숫자형 (NaN 포함)으로 파싱하는 함수 ---
def process_horsepower(hp_str):
    # Pandas의 NaN, Python의 None, NumPy의 np.nan 등 모든 형태의 결측치와 문자열이 아닌 값 처리
    if pd.isna(hp_str) or not isinstance(hp_str, str):
        return np.nan

    # 'unknown', 'n/a', 'na', 빈 문자열 등을 소문자로 변환 후 np.nan으로 처리
    # .strip().lower()를 먼저 적용하여 대소문자 및 앞뒤 공백 문제 해결
    if hp_str.strip().lower() in ['unknown', 'n/a', 'na', '']:
        return np.nan

    # ' HP', 'H P'와 같은 단위 제거 및 소문자 변환 후 앞뒤 공백 제거
    cleaned_str = re.sub(r'\s*hp\s*', '', hp_str.lower()).strip()

    if 'less than' in cleaned_str:
        match = re.search(r'\d+', cleaned_str) # 숫자 추출
        if match:
            return float(match.group(0)) * 0.5 # 예: Less than 150 -> 75
        else:
            return 75.0 # 숫자가 없으면 기본값
    elif re.search(r'\d+\s*[-–—]\s*\d+', cleaned_str): # 숫자 - 숫자 (다양한 대시 문자 포함)
        parts = re.split(r'[-–—]', cleaned_str)
        try:
            lower = float(parts[0].strip())
            upper = float(parts[1].strip())
            return (lower + upper) / 2
        except (ValueError, IndexError):
            return np.nan # 파싱 오류 시 NaN
    elif '+' in cleaned_str: # '900+' 같은 경우
        match = re.search(r'\d+', cleaned_str) # 숫자 추출
        if match:
            return float(match.group(0)) + 50.0 
        else:
            return np.nan
    else: 
        try:
            return float(cleaned_str)
        except ValueError:
            return np.nan # 최종 변환 실패 시 NaN

# 'horsepower' 컬럼에 함수 적용
if 'horsepower' in car_df.columns:
    print("\n알림: 'horsepower' 컬럼 전처리 시작 (문자열 -> 숫자 변환).")
    car_df['horsepower'] = car_df['horsepower'].apply(process_horsepower)
    print("알림: 'horsepower' 컬럼 문자열 -> 숫자 변환 완료.")

# horsepower 전처리 (문자열 -> 숫자 변환) 후 상태 확인
# 이 단계에서는 'Unknown'이 np.nan으로 변환되어 보일 것입니다.
print("\n--- horsepower 전처리 (문자열 -> 숫자 변환) 후 car_df['horsepower'] 상태 (NaN 포함) ---")
print(car_df['horsepower'].head(15))
print(car_df['horsepower'].info())
print("NaN 값 개수 (process_horsepower 적용 후):", car_df['horsepower'].isna().sum())


# --- 2. 'horsepower' 컬럼의 결측치(NaN)를 중앙값으로 채우기 ---
if 'horsepower' in car_df.columns:
    print("\n알림: 'horsepower' 컬럼의 결측치(NaN)를 중앙값으로 채웁니다.")
    # 이 단계에서 'Unknown'에서 변환된 NaN 값들이 중앙값으로 채워집니다.
    imputer = SimpleImputer(strategy='median')
    # imputer는 2D 배열을 기대하므로 car_df[['horsepower']]와 같이 DataFrame 형태로 전달합니다.
    car_df['horsepower'] = imputer.fit_transform(car_df[['horsepower']])
    print("알림: 'horsepower' 컬럼 결측치 처리 완료.")

# horsepower 전처리 (결측치 처리까지) 완료 후 최종 상태 확인
# 이 단계에서는 'Unknown'이었던 값들이 이제 중앙값으로 채워져 있을 것입니다.
print("\n--- horsepower 전처리 (결측치 처리 포함) 완료 후 car_df['horsepower'] 최종 상태 ---")
print(car_df['horsepower'].head(15))
print(car_df['horsepower'].info())
print("NaN 값 개수 (최종 확인):", car_df['horsepower'].isna().sum())

print("\n--- horsepower 컬럼에 대한 모든 전처리가 완료되었습니다. ---")

--- 전처리 전 car_df['horsepower'] 상태 ---
0              Unknown
1         400 - 500 HP
2         400 - 500 HP
3         600 - 700 HP
4     Less than 150 HP
5         200 - 300 HP
6              Unknown
7              Unknown
8         200 - 300 HP
9         200 - 300 HP
10        500 - 600 HP
11        500 - 600 HP
12        300 - 400 HP
13    Less than 150 HP
14             Unknown
Name: horsepower, dtype: object
<class 'pandas.core.series.Series'>
RangeIndex: 9970 entries, 0 to 9969
Series name: horsepower
Non-Null Count  Dtype 
--------------  ----- 
9970 non-null   object
dtypes: object(1)
memory usage: 78.0+ KB
None

알림: 'horsepower' 컬럼 전처리 시작 (문자열 -> 숫자 변환).
알림: 'horsepower' 컬럼 문자열 -> 숫자 변환 완료.

--- horsepower 전처리 (문자열 -> 숫자 변환) 후 car_df['horsepower'] 상태 (NaN 포함) ---
0       NaN
1     450.0
2     450.0
3     650.0
4      75.0
5     250.0
6       NaN
7       NaN
8     250.0
9     250.0
10    550.0
11    550.0
12    350.0
13     75.0
14      NaN
Name: horsepower, dtype: float64
<class

In [158]:
""" import re
# 이 함수는 문자열 형태의 마력 값을 파싱하여 숫자형으로 반환합니다.
# 'Unknown', 'Less than X HP', 'A - B HP', 'X+ HP', 순수 숫자 등 다양한 케이스를 처리합니다.
def process_horsepower(hp_str):
    # NaN이거나 문자열이 아니면 바로 NaN 반환 (Pandas의 NaN, None, np.nan 등 포함)
    if pd.isna(hp_str) or not isinstance(hp_str, str):
        return np.nan

    # 추가적인 결측치 문자열 처리: 'N/A', 빈 문자열 등을 np.nan으로 변환
    if hp_str.strip().lower() in ['unknown', 'n/a', 'na', '']:
        return np.nan

    # 모든 공백, 'HP', 'H P' 등을 제거하고 소문자로 변환하여 일관성 유지
    cleaned_str = re.sub(r'\s*hp\s*', '', hp_str.lower()).strip()

    if 'less than' in cleaned_str:
        match = re.search(r'\d+', cleaned_str) # 숫자 추출
        if match:
            return float(match.group(0)) * 0.5 # 예: Less than 150 -> 75
        else:
            return 75.0 # 숫자가 없으면 기본값
    elif re.search(r'\d+\s*[-–—]\s*\d+', cleaned_str): # 숫자 - 숫자 (다양한 대시 문자 포함)
        parts = re.split(r'[-–—]', cleaned_str)
        try:
            lower = float(parts[0].strip())
            upper = float(parts[1].strip())
            return (lower + upper) / 2
        except (ValueError, IndexError):
            return np.nan # 파싱 오류 시 NaN
    elif '+' in cleaned_str: # '900+' 같은 경우
        match = re.search(r'\d+', cleaned_str) # 숫자 추출
        if match:
            return float(match.group(0)) + 50.0 # 예: 900+ -> 950 (임의의 상한값)
        else:
            return np.nan
    else: # 순수한 숫자 형태의 문자열 (예: '100') 또는 예상치 못한 다른 형태
        try:
            return float(cleaned_str)
        except ValueError:
            return np.nan # 최종 변환 실패 시 NaN

# 'horsepower' 컬럼에 함수 적용
if 'horsepower' in car_df.columns:
    print("알림: 'horsepower' 컬럼 전처리 시작 (문자열 -> 숫자 변환).")
    car_df['horsepower'] = car_df['horsepower'].apply(process_horsepower)
    print("알림: 'horsepower' 컬럼 문자열 -> 숫자 변환 완료.")

# horsepower 전처리 (문자열 -> 숫자 변환) 후 상태 확인
print("\n--- horsepower 전처리 (문자열 -> 숫자 변환) 후 car_df['horsepower'] 상태 ---")
print(car_df['horsepower'].head(25)) # 좀 더 많은 행 확인
print(car_df['horsepower'].info())
print("NaN 값 개수 (process_horsepower 적용 후):", car_df['horsepower'].isna().sum())

# 2. 'horsepower' 컬럼의 결측치(NaN)를 중앙값으로 채우기
if 'horsepower' in car_df.columns:
    print("\n알림: 'horsepower' 컬럼의 결측치(NaN)를 중앙값으로 채웁니다.")
    # SimpleImputer를 사용하여 중앙값으로 채우기
    # imputer는 2D 배열을 기대하므로 reshape(-1, 1) 해줍니다.
    imputer = SimpleImputer(strategy='median')
    car_df['horsepower'] = imputer.fit_transform(car_df[['horsepower']])
    print("알림: 'horsepower' 컬럼 결측치 처리 완료.")

# horsepower 전처리 (결측치 처리까지) 완료 후 최종 상태 확인
print("\n--- horsepower 전처리 (결측치 처리 포함) 완료 후 car_df['horsepower'] 최종 상태 ---")
print(car_df['horsepower'].head(25)) # 최종 상태 확인
print(car_df['horsepower'].info())
print("NaN 값 개수 (최종 확인):", car_df['horsepower'].isna().sum())

print("\n--- horsepower 컬럼에 대한 모든 전처리가 완료되었습니다. ---") """

' import re\n# 이 함수는 문자열 형태의 마력 값을 파싱하여 숫자형으로 반환합니다.\n# \'Unknown\', \'Less than X HP\', \'A - B HP\', \'X+ HP\', 순수 숫자 등 다양한 케이스를 처리합니다.\ndef process_horsepower(hp_str):\n    # NaN이거나 문자열이 아니면 바로 NaN 반환 (Pandas의 NaN, None, np.nan 등 포함)\n    if pd.isna(hp_str) or not isinstance(hp_str, str):\n        return np.nan\n\n    # 추가적인 결측치 문자열 처리: \'N/A\', 빈 문자열 등을 np.nan으로 변환\n    if hp_str.strip().lower() in [\'unknown\', \'n/a\', \'na\', \'\']:\n        return np.nan\n\n    # 모든 공백, \'HP\', \'H P\' 등을 제거하고 소문자로 변환하여 일관성 유지\n    cleaned_str = re.sub(r\'\\s*hp\\s*\', \'\', hp_str.lower()).strip()\n\n    if \'less than\' in cleaned_str:\n        match = re.search(r\'\\d+\', cleaned_str) # 숫자 추출\n        if match:\n            return float(match.group(0)) * 0.5 # 예: Less than 150 -> 75\n        else:\n            return 75.0 # 숫자가 없으면 기본값\n    elif re.search(r\'\\d+\\s*[-–—]\\s*\\d+\', cleaned_str): # 숫자 - 숫자 (다양한 대시 문자 포함)\n        parts = re.split(r\'[-–—]\', cleaned_str)\n        try:\n     

In [159]:
X = car_df.drop('price_in_aed', axis=1) # 'price_in_aed'를 제외한 나머지 컬럼이 X가 됩니다.
X

,kilometers,body_condition,mechanical_condition,seller_type,body_type,no_of_cylinders,transmission_type,regional_specs,horsepower,fuel_type,year,color,emirate,company,date_posted
0,167390,Perfect inside and out,Perfect inside and out,Dealer,SUV,6,Automatic Transmission,GCC Specs,350.0,Gasoline,2013.0,Silver,Dubai,mitsubishi,13/05/2022
1,39000,Perfect inside and out,Perfect inside and out,Dealer,SUV,8,Automatic Transmission,North American Specs,450.0,Gasoline,2018.0,White,Sharjah,chevrolet,14/01/2022
2,200000,Perfect inside and out,Perfect inside and out,Dealer,Sedan,6,Automatic Transmission,GCC Specs,450.0,Gasoline,2014.0,Blue,Sharjah,mercedes-benz,05/05/2022
3,27000,Perfect inside and out,Perfect inside and out,Dealer,Hard Top Convertible,8,Automatic Transmission,GCC Specs,650.0,Gasoline,2018.0,Red,Dubai,ferrari,30/04/2022
4,69000,Perfect inside and out,Perfect inside and out,Owner,Wagon,4,Manual Transmission,GCC Specs,75.0,Gasoline,2020.0,White,Dubai,renault,13/05/2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9965,105777,Perfect inside and out,Perfect inside and out,Dealer,SUV,8,Automatic Transmission,GCC Specs,550.0,Gasoline,2015.0,White,Dubai,land-rover,18/11/2021
9966,55640,Perfect inside and out,Perfect inside and out,Owner,SUV,8,Automatic Transmission,GCC Specs,450.0,Gasoline,2014.0,White,Dubai,cadillac,11/05/2022
9967,100,Perfect inside and out,Perfect inside and out,Dealer,SUV,8,Automatic Transmission,Other,450.0,Gasoline,NaN,Black,Dubai,land-rover,14/12/2021
9968,140000,"No accidents, very few faults",Perfect inside and out,Owner,Sedan,4,Automatic Transmission,GCC Specs,175.0,Gasoline,2013.0,White,Dubai,chevrolet,06/03/2022


In [ ]:
from datetime import datetime

if 'date_posted ' in X.columns:
    X['date_posted_dt'] = pd.to_datetime(X['date_posted '], format='%d/%m/%Y', errors='coerce')
    print("\n알림: 'date_posted ' 컬럼이 datetime 객체로 변환되었습니다 ('date_posted_dt' 생성).")
else:
    print("\n경고: 'date_posted ' 컬럼이 X 데이터프레임에 없습니다.")

# 변환 후 상태 확인
print("\n--- 'date_posted_dt' 컬럼 정보 ---")
print(X['date_posted_dt'].info())
print(X['date_posted_dt'].head())


# --- 2. datetime 객체에서 유용한 숫자형 특성 파생 ---

# 예를 들어, X['date_posted_dt'].max()를 사용할 수도 있습니다.
reference_date = datetime(2023, 12, 31) # 예시: 2023년 12월 31일을 기준 날짜로 설정

if 'date_posted_dt' in X.columns:
    # 2.1 게시일로부터 경과 일수 (Days since posted)
    # NaT 값은 계산 시 NaN이 됩니다.
    X['days_since_posted'] = (reference_date - X['date_posted_dt']).dt.days
    print("알림: 'days_since_posted' 컬럼이 생성되었습니다.")

    # 2.2 게시 월 (Month of posting)
    X['post_month'] = X['date_posted_dt'].dt.month
    print("알림: 'post_month' 컬럼이 생성되었습니다.")

    # 2.3 게시 요일 (Day of week, 월=0, 일=6)
    X['post_day_of_week'] = X['date_posted_dt'].dt.dayofweek
    print("알림: 'post_day_of_week' 컬럼이 생성되었습니다.")

    # 'year' 컬럼이 차량 제조 연도라고 가정합니다.
    if 'year' in X.columns:
        X['age_at_post'] = X['date_posted_dt'].dt.year - X['year']
        # 만약 date_posted_dt.dt.year가 NaT로 인해 NaN이면 age_at_post도 NaN이 됩니다.
        X['age_at_post'] = X['age_at_post'].apply(lambda x: max(0, x) if pd.notna(x) else np.nan) # 나이가 음수가 되지 않도록 0으로 처리
        print("알림: 'age_at_post' 컬럼이 생성되었습니다.")
    else:
        print("경고: 'year' 컬럼이 없어 'age_at_post'를 생성할 수 없습니다.")

    # 2.5 게시 연도 (Post year) - 필요시
    X['post_year'] = X['date_posted_dt'].dt.year
    print("알림: 'post_year' 컬럼이 생성되었습니다.")


    # --- 3. 원본 'date_posted ' 및 중간 'date_posted_dt' 컬럼 삭제 ---
    X = X.drop(columns=['date_posted ', 'date_posted_dt'], errors='ignore')
    print("\n알림: 원본 'date_posted ' 및 중간 'date_posted_dt' 컬럼이 삭제되었습니다.")
else:
    print("\n경고: 'date_posted_dt' 컬럼이 없어 날짜 특성을 파생할 수 없습니다.")


# --- 4. 파생된 특성들이 추가된 X 데이터프레임 최종 확인 ---
print("\n--- 날짜 특성 공학 후 X 데이터프레임의 상위 5행 ---")
print(X.head())
print("\n--- 날짜 특성 공학 후 X 데이터프레임 정보 ---")
print(X.info())



알림: 'date_posted ' 컬럼이 datetime 객체로 변환되었습니다 ('date_posted_dt' 생성).

--- 'date_posted_dt' 컬럼 정보 ---
<class 'pandas.core.series.Series'>
RangeIndex: 9970 entries, 0 to 9969
Series name: date_posted_dt
Non-Null Count  Dtype         
--------------  -----         
9970 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 78.0 KB
None
0   2022-05-13
1   2022-01-14
2   2022-05-05
3   2022-04-30
4   2022-05-13
Name: date_posted_dt, dtype: datetime64[ns]
알림: 'days_since_posted' 컬럼이 생성되었습니다.
알림: 'post_month' 컬럼이 생성되었습니다.
알림: 'post_day_of_week' 컬럼이 생성되었습니다.
알림: 'age_at_post' 컬럼이 생성되었습니다.
알림: 'post_year' 컬럼이 생성되었습니다.

알림: 원본 'date_posted ' 및 중간 'date_posted_dt' 컬럼이 삭제되었습니다.

--- 날짜 특성 공학 후 X 데이터프레임의 상위 5행 ---
   kilometers          body_condition    mechanical_condition seller_type  \
0      167390  Perfect inside and out  Perfect inside and out      Dealer   
1       39000  Perfect inside and out  Perfect inside and out      Dealer   
2      200000  Perfect inside and out  Perfect 

In [ ]:

print("--- 전처리 전 X 데이터프레임의 'no_of_cylinders' 정보 ---")
print(X['no_of_cylinders'].info())
print(X['no_of_cylinders'].head(10)) # 더 많은 샘플 확인


# --- 1. 'no_of_cylinders' 컬럼의 비정상 문자열을 NaN으로 변환 ---
if 'no_of_cylinders' in X.columns:
    print("\n알림: 'no_of_cylinders' 컬럼의 비정상 문자열을 NaN으로 변환합니다.")
    # Step 1.1: 컬럼을 문자열 타입으로 명시적으로 변환하여 .str accessor가 항상 작동하도록 합니다.
    # 이 과정에서 np.nan은 문자열 'nan'으로 변환됩니다.
    X['no_of_cylinders'] = X['no_of_cylinders'].astype(str)

    # Step 1.2: 대소문자 문제를 해결하기 위해 replace() 전에 .str.lower()를 적용합니다.
    X['no_of_cylinders'] = X['no_of_cylinders'].str.lower().replace(
        ['unknown', 'null', 'nome', 'none', '', 'nan'], 
        np.nan, # 대체될 값
        regex=False # 정규표현식 사용 안 함
    )
    print("알림: 비정상 문자열 NaN 변환 완료.")

    # 2. 'no_of_cylinders' 컬럼을 숫자형으로 강제 변환
    # 이제 문자열이 NaN으로 변환되었으므로, pd.to_numeric을 안전하게 적용합니다.
    try:
        X['no_of_cylinders'] = pd.to_numeric(X['no_of_cylinders'], errors='coerce')
        print("알림: 'no_of_cylinders' 컬럼이 숫자형으로 변환되었습니다.")
    except Exception as e:
        print(f"경고: 'no_of_cylinders' 컬럼을 숫자형으로 변환하는 데 실패했습니다: {e}")

    # 3. 'no_of_cylinders' 컬럼의 결측치(NaN)를 중앙값으로 채우기
    # SimpleImputer를 사용하여 중앙값으로 채웁니다.
    print("\n알림: 'no_of_cylinders' 컬럼의 결측치(NaN)를 중앙값으로 채웁니다.")
    imputer = SimpleImputer(strategy='median')
    X['no_of_cylinders'] = imputer.fit_transform(X[['no_of_cylinders']])
    print("알림: 'no_of_cylinders' 컬럼 결측치 처리 완료.")

else:
    print("\n알림: 'no_of_cylinders' 컬럼이 X 데이터프레임에 없습니다. 전처리할 필요가 없습니다.")


# --- 4. 변환 후 'no_of_cylinders' 컬럼 최종 상태 확인 ---
print("\n--- 전처리 완료 후 X 데이터프레임의 'no_of_cylinders' 정보 ---")
print(X['no_of_cylinders'].info())
print(X['no_of_cylinders'].head(10)) # 최종 상태 확인
print(f"\n최종 'no_of_cylinders' 컬럼의 NaN 개수: {X['no_of_cylinders'].isna().sum()}")

print("\n--- 'no_of_cylinders' 컬럼에 대한 모든 전처리가 완료되었습니다. ---")


--- 전처리 전 X 데이터프레임의 'no_of_cylinders' 정보 ---
<class 'pandas.core.series.Series'>
RangeIndex: 9970 entries, 0 to 9969
Series name: no_of_cylinders
Non-Null Count  Dtype 
--------------  ----- 
9889 non-null   object
dtypes: object(1)
memory usage: 78.0+ KB
None
0    6
1    8
2    6
3    8
4    4
5    6
6    8
7    4
8    6
9    6
Name: no_of_cylinders, dtype: object

알림: 'no_of_cylinders' 컬럼의 비정상 문자열을 NaN으로 변환합니다.
알림: 비정상 문자열 NaN 변환 완료.
알림: 'no_of_cylinders' 컬럼이 숫자형으로 변환되었습니다.

알림: 'no_of_cylinders' 컬럼의 결측치(NaN)를 중앙값으로 채웁니다.
알림: 'no_of_cylinders' 컬럼 결측치 처리 완료.

--- 전처리 완료 후 X 데이터프레임의 'no_of_cylinders' 정보 ---
<class 'pandas.core.series.Series'>
RangeIndex: 9970 entries, 0 to 9969
Series name: no_of_cylinders
Non-Null Count  Dtype  
--------------  -----  
9970 non-null   float64
dtypes: float64(1)
memory usage: 78.0 KB
None
0    6.0
1    8.0
2    6.0
3    8.0
4    4.0
5    6.0
6    8.0
7    4.0
8    6.0
9    6.0
Name: no_of_cylinders, dtype: float64

최종 'no_of_cylinders' 컬럼의 NaN 개수: 0

-

In [162]:
X

,kilometers,body_condition,mechanical_condition,seller_type,body_type,no_of_cylinders,transmission_type,regional_specs,horsepower,fuel_type,year,color,emirate,company,days_since_posted,post_month,post_day_of_week,age_at_post,post_year
0,167390,Perfect inside and out,Perfect inside and out,Dealer,SUV,6.0,Automatic Transmission,GCC Specs,350.0,Gasoline,2013.0,Silver,Dubai,mitsubishi,597,5,4,9.0,2022
1,39000,Perfect inside and out,Perfect inside and out,Dealer,SUV,8.0,Automatic Transmission,North American Specs,450.0,Gasoline,2018.0,White,Sharjah,chevrolet,716,1,4,4.0,2022
2,200000,Perfect inside and out,Perfect inside and out,Dealer,Sedan,6.0,Automatic Transmission,GCC Specs,450.0,Gasoline,2014.0,Blue,Sharjah,mercedes-benz,605,5,3,8.0,2022
3,27000,Perfect inside and out,Perfect inside and out,Dealer,Hard Top Convertible,8.0,Automatic Transmission,GCC Specs,650.0,Gasoline,2018.0,Red,Dubai,ferrari,610,4,5,4.0,2022
4,69000,Perfect inside and out,Perfect inside and out,Owner,Wagon,4.0,Manual Transmission,GCC Specs,75.0,Gasoline,2020.0,White,Dubai,renault,597,5,4,2.0,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9965,105777,Perfect inside and out,Perfect inside and out,Dealer,SUV,8.0,Automatic Transmission,GCC Specs,550.0,Gasoline,2015.0,White,Dubai,land-rover,773,11,3,6.0,2021
9966,55640,Perfect inside and out,Perfect inside and out,Owner,SUV,8.0,Automatic Transmission,GCC Specs,450.0,Gasoline,2014.0,White,Dubai,cadillac,599,5,2,8.0,2022
9967,100,Perfect inside and out,Perfect inside and out,Dealer,SUV,8.0,Automatic Transmission,Other,450.0,Gasoline,NaN,Black,Dubai,land-rover,747,12,1,NaN,2021
9968,140000,"No accidents, very few faults",Perfect inside and out,Owner,Sedan,4.0,Automatic Transmission,GCC Specs,175.0,Gasoline,2013.0,White,Dubai,chevrolet,665,3,6,9.0,2022


In [163]:
# --- 1. 원-핫 인코딩할 컬럼 정의 ---
# 'body_condition'과 'mechanical_conditions'만 원-핫 인코딩합니다.
columns_to_one_hot_encode = [
    'body_condition',
    'mechanical_condition',
    'seller_type',
    'body_type',
    'transmission_type',
    'regional_specs',
    'fuel_type',
    'emirate',
    'color',
    'company'
]

# 실제로 X에 존재하는 컬럼만 선택 (안전 장치)
columns_to_one_hot_encode = [col for col in columns_to_one_hot_encode if col in X.columns]

print(f"\n원-핫 인코딩 대상 컬럼: {columns_to_one_hot_encode}")


X_processed = pd.get_dummies(X, columns=columns_to_one_hot_encode, drop_first=False)



원-핫 인코딩 대상 컬럼: ['body_condition', 'mechanical_condition', 'seller_type', 'body_type', 'transmission_type', 'regional_specs', 'fuel_type', 'emirate', 'color', 'company']


In [164]:
X_processed

,kilometers,no_of_cylinders,horsepower,year,days_since_posted,post_month,post_day_of_week,age_at_post,post_year,"body_condition_A bit of wear & tear, all repaired",...,company_skoda,company_smart,company_ssang-yong,company_subaru,company_suzuki,company_tesla,company_toyota,company_volkswagen,company_volvo,company_westfield-sportscars
0,167390,6.0,350.0,2013.0,597,5,4,9.0,2022,False,...,False,False,False,False,False,False,False,False,False,False
1,39000,8.0,450.0,2018.0,716,1,4,4.0,2022,False,...,False,False,False,False,False,False,False,False,False,False
2,200000,6.0,450.0,2014.0,605,5,3,8.0,2022,False,...,False,False,False,False,False,False,False,False,False,False
3,27000,8.0,650.0,2018.0,610,4,5,4.0,2022,False,...,False,False,False,False,False,False,False,False,False,False
4,69000,4.0,75.0,2020.0,597,5,4,2.0,2022,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9965,105777,8.0,550.0,2015.0,773,11,3,6.0,2021,False,...,False,False,False,False,False,False,False,False,False,False
9966,55640,8.0,450.0,2014.0,599,5,2,8.0,2022,False,...,False,False,False,False,False,False,False,False,False,False
9967,100,8.0,450.0,NaN,747,12,1,NaN,2021,False,...,False,False,False,False,False,False,False,False,False,False
9968,140000,4.0,175.0,2013.0,665,3,6,9.0,2022,False,...,False,False,False,False,False,False,False,False,False,False


In [165]:

print(car_df.columns.tolist())
car_df

['price_in_aed', 'kilometers', 'body_condition', 'mechanical_condition', 'seller_type', 'body_type', 'no_of_cylinders', 'transmission_type', 'regional_specs', 'horsepower', 'fuel_type', 'year', 'color', 'emirate', 'company', 'date_posted ']


,price_in_aed,kilometers,body_condition,mechanical_condition,seller_type,body_type,no_of_cylinders,transmission_type,regional_specs,horsepower,fuel_type,year,color,emirate,company,date_posted
0,26000,167390,Perfect inside and out,Perfect inside and out,Dealer,SUV,6,Automatic Transmission,GCC Specs,350.0,Gasoline,2013.0,Silver,Dubai,mitsubishi,13/05/2022
1,110000,39000,Perfect inside and out,Perfect inside and out,Dealer,SUV,8,Automatic Transmission,North American Specs,450.0,Gasoline,2018.0,White,Sharjah,chevrolet,14/01/2022
2,78000,200000,Perfect inside and out,Perfect inside and out,Dealer,Sedan,6,Automatic Transmission,GCC Specs,450.0,Gasoline,2014.0,Blue,Sharjah,mercedes-benz,05/05/2022
3,899000,27000,Perfect inside and out,Perfect inside and out,Dealer,Hard Top Convertible,8,Automatic Transmission,GCC Specs,650.0,Gasoline,2018.0,Red,Dubai,ferrari,30/04/2022
4,33000,69000,Perfect inside and out,Perfect inside and out,Owner,Wagon,4,Manual Transmission,GCC Specs,75.0,Gasoline,2020.0,White,Dubai,renault,13/05/2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9965,215000,105777,Perfect inside and out,Perfect inside and out,Dealer,SUV,8,Automatic Transmission,GCC Specs,550.0,Gasoline,2015.0,White,Dubai,land-rover,18/11/2021
9966,90000,55640,Perfect inside and out,Perfect inside and out,Owner,SUV,8,Automatic Transmission,GCC Specs,450.0,Gasoline,2014.0,White,Dubai,cadillac,11/05/2022
9967,679000,100,Perfect inside and out,Perfect inside and out,Dealer,SUV,8,Automatic Transmission,Other,450.0,Gasoline,NaN,Black,Dubai,land-rover,14/12/2021
9968,18900,140000,"No accidents, very few faults",Perfect inside and out,Owner,Sedan,4,Automatic Transmission,GCC Specs,175.0,Gasoline,2013.0,White,Dubai,chevrolet,06/03/2022


In [166]:
string_columns_in_processed_X = X_processed.select_dtypes(include='object').columns.tolist()

if string_columns_in_processed_X:
    print(f"\n경고: X_processed에 다음 문자열(object) 타입 컬럼이 남아있습니다 ({len(string_columns_in_processed_X)}개):")
    for col in string_columns_in_processed_X:
        print(f"- '{col}' (샘플 값: {X_processed[col].iloc[0]})") # 첫 번째 값도 함께 출력
    print("\n선형 회귀 모델은 문자열 데이터를 처리할 수 없으므로, 이 컬럼들을 추가 전처리해야 합니다.")
else:
    print("\n알림: X_processed에 문자열(object) 타입 컬럼이 없습니다. 모든 특성이 숫자형입니다.")



알림: X_processed에 문자열(object) 타입 컬럼이 없습니다. 모든 특성이 숫자형입니다.


In [167]:

# --- 디버깅: train_test_split 직전 X_processed의 결측치 확인 ---
print("\n--- 디버깅: train_test_split 직전 X_processed의 결측치 개수 ---")
nan_counts = X_processed.isnull().sum()
print(nan_counts[nan_counts > 0]) # NaN이 있는 컬럼만 출력

if nan_counts.sum() > 0:
    print("\n경고: X_processed에 결측치(NaN)가 남아있습니다. 모델 훈련 전에 처리해야 합니다.")
    # 결측치 처리 (SimpleImputer)를 다시 적용하거나, ColumnTransformer가 제대로 적용되었는지 확인해야 합니다.
    # 가장 간단한 방법은 전체 X_processed에 대해 SimpleImputer를 다시 적용하는 것입니다.
    from sklearn.impute import SimpleImputer
    imputer_final = SimpleImputer(strategy='median')
    # 모든 수치형 컬럼에 대해 imputer 적용
    # 주의: X_processed에 문자열 컬럼이 있다면 오류가 발생할 수 있습니다.
    #       이전 단계에서 모든 문자열이 숫자형으로 변환되었어야 합니다.
    numerical_cols_in_X_processed = X_processed.select_dtypes(include=np.number).columns
    X_processed[numerical_cols_in_X_processed] = imputer_final.fit_transform(X_processed[numerical_cols_in_X_processed])
    print("\n알림: X_processed의 남은 결측치를 중앙값으로 채웠습니다.")
    print("--- 결측치 채운 후 X_processed의 결측치 개수 ---")
    print(X_processed.isnull().sum()[X_processed.isnull().sum() > 0])
else:
    print("\n알림: X_processed에 결측치(NaN)가 없습니다. 모델 훈련을 진행합니다.")


--- 디버깅: train_test_split 직전 X_processed의 결측치 개수 ---
year           970
age_at_post    970
dtype: int64

경고: X_processed에 결측치(NaN)가 남아있습니다. 모델 훈련 전에 처리해야 합니다.

알림: X_processed의 남은 결측치를 중앙값으로 채웠습니다.
--- 결측치 채운 후 X_processed의 결측치 개수 ---
Series([], dtype: int64)


In [168]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=156)

In [169]:
lr = LinearRegression()
lr.fit(X_train, y_train)

# 예측
y_pred = lr.predict(X_test)
print(y_pred[:3])

[ 82522.25964367 579666.2114456  128454.79061133]


In [170]:
mse = mean_squared_error(y_test, y_pred)
rmse= np.sqrt(mse)
mse, rmse

(np.float64(147743269256.17212), np.float64(384373.86651042255))

In [171]:
r2_score(y_test, y_pred)

np.float64(0.5102582177359831)

In [172]:

evals = [(X_test, y_test)]
xgb = XGBRegressor(n_estimators=400, learning_rate=0.1, max_depth=3, use_label_encoder=False)
xgb.fit(X_train, y_train,  eval_set=evals, verbose=True)
xgb_pred = xgb.predict(X_test)

[0]	validation_0-rmse:576471.51329
[1]	validation_0-rmse:549505.02059
[2]	validation_0-rmse:525534.86337
[3]	validation_0-rmse:506008.01677
[4]	validation_0-rmse:488031.25104
[5]	validation_0-rmse:472359.35633
[6]	validation_0-rmse:458083.89907
[7]	validation_0-rmse:447326.47562
[8]	validation_0-rmse:434462.90950
[9]	validation_0-rmse:425194.73644
[10]	validation_0-rmse:418019.84098
[11]	validation_0-rmse:408926.07775
[12]	validation_0-rmse:403041.71838
[13]	validation_0-rmse:398411.52438
[14]	validation_0-rmse:393457.73228
[15]	validation_0-rmse:388715.45979
[16]	validation_0-rmse:384240.22403
[17]	validation_0-rmse:380886.59258
[18]	validation_0-rmse:375910.49280
[19]	validation_0-rmse:373084.20694
[20]	validation_0-rmse:371183.97784
[21]	validation_0-rmse:369365.51038
[22]	validation_0-rmse:362857.40990
[23]	validation_0-rmse:360777.62537
[24]	validation_0-rmse:358800.38646
[25]	validation_0-rmse:357610.84590
[26]	validation_0-rmse:356764.66679
[27]	validation_0-rmse:355213.23966
[2

In [173]:
xgb_pred[:3]

array([ 98949.75, 558123.94, 211960.95], dtype=float32)

In [174]:
mse = mean_squared_error(y_test, xgb_pred)
rmse= np.sqrt(mse)
mse, rmse

(np.float64(108619851045.9352), np.float64(329575.25854641333))

In [175]:
r2_score(y_test, xgb_pred)

np.float64(0.6399451581902366)

In [176]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=156)

In [177]:


rf = RandomForestRegressor(random_state=100, max_depth=3)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
y_pred[:3]

array([ 89302.144331  , 416162.90271688, 236131.24973528])

In [178]:
mse = mean_squared_error(y_test, y_pred)
rmse= np.sqrt(mse)
mse, rmse

(np.float64(156683585049.20377), np.float64(395832.7740968448))

In [179]:
r2_score(y_test, y_pred)

np.float64(0.48062271411861923)

In [145]:
r2_scores= cross_val_score(rf, X_processed, y, scoring='r2', cv=5)
r2_scores, np.mean(r2_scores)

(array([0.6071816 , 0.63693333, 0.45794819, 0.63774515, 0.7460201 ]),
 np.float64(0.6171656758802064))

In [180]:
results = []

X_train, X_test, y_train, y_test = train_test_split(X_processed,y, test_size=0.3, random_state=42)

In [181]:
for degree in range(1,2):
  model_poly = Pipeline([
    ('poly', PolynomialFeatures(degree=degree, include_bias=False)),
    ('linear', LinearRegression())]
  )
  model_poly.fit(X_train, y_train)
  pred_poly = model_poly.predict(X_test)
  mse = mean_squared_error(y_test, pred_poly)
  rmse = np.sqrt(mse)
  r2 = r2_score(y_test, pred_poly)
  
  results.append({ 'degree':degree,
                  'MSE':mse,
                  'RMSE':rmse,
                  'R2': r2})

pd.DataFrame(results)



,degree,MSE,RMSE,R2
0,1,1.176364e+11,342981.598722,0.53342


In [182]:
r2_score(y_test, pred_poly)

np.float64(0.5334202916163552)

# 다항연산

In [183]:
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
pred_ridge = ridge.predict(X_test)

mse = mean_squared_error(y_test, pred_ridge)
r2  = r2_score(y_test, pred_ridge)
mse, r2

c:\Users\Admin\miniconda3\envs\hi_ml_env\lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=6.69733e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


(np.float64(117919712831.01497), np.float64(0.5322965004569333))

In [184]:
alphas = [0.001, 0.01, 0.1, 1, 10, 100]
ridge_cv = RidgeCV(alphas=alphas, cv=5)

In [185]:
ridge_cv.fit(X_train, y_train)
ridge_preds = ridge_cv.predict(X_test)

c:\Users\Admin\miniconda3\envs\hi_ml_env\lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=2.47238e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\Admin\miniconda3\envs\hi_ml_env\lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=7.54695e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\Admin\miniconda3\envs\hi_ml_env\lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=7.54529e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\Admin\miniconda3\envs\hi_ml_env\lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=7.54685e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\Adm

In [186]:
ridge_mse = mean_squared_error(y_test, ridge_preds)
ridge_r2 = r2_score(y_test, ridge_preds)
print(f'ridge cv mse: {ridge_mse:4f}, r2 : {ridge_r2:4f}')

ridge cv mse: 117636578145.366928, r2 : 0.533419


# 릿지

In [187]:
lasso = Lasso(alpha=0.1)  # alpha 값 작으면 규제 약해짐
lasso.fit(X_train, y_train)
pred_lasso = lasso.predict(X_test)

print("\n[라쏘 회귀]")
print("MSE:", mean_squared_error(y_test, pred_lasso))
print("R2:", r2_score(y_test, pred_lasso))


[라쏘 회귀]
MSE: 118004248028.54175
R2: 0.531961209547817


c:\Users\Admin\miniconda3\envs\hi_ml_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.021e+14, tolerance: 1.298e+11
  model = cd_fast.enet_coordinate_descent(


In [188]:
lasso_cv = LassoCV(alphas=alphas, cv=5, max_iter=10000)
lasso_cv.fit(X_train, y_train)
lasso_preds = lasso_cv.predict(X_test)
lasso_mse = mean_squared_error(y_test, lasso_preds)
lasso_r2 = r2_score(y_test, lasso_preds)

c:\Users\Admin\miniconda3\envs\hi_ml_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26223654645575.22, tolerance: 107552282513.58269
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\Admin\miniconda3\envs\hi_ml_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 125883126590091.05, tolerance: 107552282513.58269
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\Admin\miniconda3\envs\hi_ml_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 134002365671143.75, tolerance: 107552282513.58269
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\Admin\miniconda3\envs\hi_ml_env\

In [189]:
print(f"[개선 Lasso] 최적 alpha: {lasso_cv.alpha_}")
print(f"MSE: {lasso_mse:.3f}, R2: {lasso_r2:.3f}")

[개선 Lasso] 최적 alpha: 10.0
MSE: 117616884570.052, R2: 0.533


# 로쏘회귀

In [190]:
enet = ElasticNet(alpha=0.1, l1_ratio=0.5)
enet.fit(X_train, y_train)

c:\Users\Admin\miniconda3\envs\hi_ml_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.292e+14, tolerance: 1.298e+11
  model = cd_fast.enet_coordinate_descent(


ElasticNet(alpha=0.1)

In [191]:
enet_pred = enet.predict(X_test)
print("\n[엘라스틱넷 회귀]")
print("MSE:", mean_squared_error(y_test, enet_pred))
print("R2:", r2_score(y_test, enet_pred))


[엘라스틱넷 회귀]
MSE: 147004874729.11743
R2: 0.4169363823059221
